# Лабораторная работа №1

Лабораторную работу необходимо выполнять в ***Google Colab***. Результат работы (в виде ссылки на notebook) выслать письмом на litvinov.vg@ssau.ru. В теме письма указывать ФИО полностью.

Для выполнения задания необходимо подобрать корпус текстов (художественных) на английском языке. Объем корпуса должен составлять не менее $3 \cdot 10^7$ символов. Далее все действия будут выполняться над выбранными данными. Для отладки алгоритмов лучше использовать не весь корпус, а лишь его часть.

In [1]:
# код загрузки корпуса
data = []
with open('./book-war-and-peace.txt', mode='r', encoding='utf-8') as file:
    for line in file:
        data.append(line)

#### Шаг №1
Заменить все числа, которые представлены цифрами, их текстовым представлением (т.е. прописью). (1 = one, 23 = twenty three, 1042 = one thousand forty two, и т.п.). Методы библиотек не использовать, алгоритм необходимо написать самостоятельно.

In [2]:
# вставьте код здесь
digit_2_text = {1:'one',
                2:'two',
                3:'three',
                4:'four',
                5:'five',
                6:'six',
                7:'seven',
                8:'eight',
                9:'nine'}

teen_2_text = { 10:'ten',
                11:'eleven',
                12:'twelve',
                13:'thirteen',
                14:'fourteen',
                15:'fifteen',
                16:'sixteen',
                17:'seventeen',
                18:'eighteen',
                19:'nineteen'}

tens_2_text =  {20:'twenty',
                30:'thirty',
                40:'fourty',
                50:'fifty',
                60:'sixty',
                70:'seventy',
                80:'eighty',
                90:'ninety'}

power_2_text = {100:'hundred',
                1000:'thousand',
                10**6:'million',
                10**9:'billion',
                10**12:'trillion',
                10**15:'quadrillion',
                10**18:'quintillion'}




In [3]:
def num2text(number):
    text = ''
    order_of_magnitude = len(str(number))
    if order_of_magnitude > 3:
        if order_of_magnitude % 3:
            text += num2text(number//10**(order_of_magnitude -(order_of_magnitude % 3))) + ' ' + power_2_text[10**(3*(order_of_magnitude//3))]
            text += num2text(number%10**(order_of_magnitude -(order_of_magnitude % 3)))
        else:
            text += num2text(number//(10**(order_of_magnitude-3))) + ' ' + power_2_text[10**(order_of_magnitude-3)]
            text += num2text(number%(10**(order_of_magnitude-3)))
    else:
        if number//100:
            text += ' ' + digit_2_text[number//100] + ' ' + power_2_text[100]
        if number%100 < 20 and number%100 > 9:
            text += ' ' + teen_2_text[number%100]
            return text
        if number%100 >= 20:
            text += ' ' + tens_2_text[10*(number%100//10)]
        if number%10:
            text += ' ' + digit_2_text[number%10]
    return text

In [4]:
num2text(999113123183412347239)

' nine hundred ninety nine quintillion one hundred thirteen quadrillion one hundred twenty three trillion one hundred eighty three billion four hundred twelve million three hundred fourty seven thousand two hundred thirty nine'

#### Шаг №2
Преобразовать текст к нижнему или верхнему регистру.

In [5]:
# вставьте код здесь
data = [line.lower() for line in data]

#### Шаг №3
Произвести токенезацию (токенами выступают слова), не учитывая знаки препинания (т.е. необходимо разбить текст на токены (лексемы)):
* С помощью nltk.tokenize.RegexpTokenizer (тут необходимо вспомнить регулярные выражения [posix](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F)).

In [6]:
# вставьте код здесь
import nltk

regtokenizer = nltk.tokenize.RegexpTokenizer(r'[A-Za-z]+')

In [7]:
tokens = []
for line in data:
    tokens += regtokenizer.tokenize(line)
tokens[:10]

['chapter', 'i', 'well', 'prince', 'so', 'genoa', 'and', 'lucca', 'are', 'now']

* С помощью nltk.word_tokenize.

In [8]:
# вставьте код здесь
wordtokens = []
for line in data:
    wordtokens += nltk.tokenize.word_tokenize(line)
wordtokens[:10]

['chapter', 'i', '``', 'well', ',', 'prince', ',', 'so', 'genoa', 'and']

#### Шаг №4
Произвести частотный анализ слов. Методы библиотек не использовать, т.е. алгоритм необходимо написать самостоятельно. Cохранить результат в выходной csv файл в порядке убывания частот. Формат выходного файла:

| Word | Count |
| --- | --- |
| cat | 1000 |
| dog | 300 |
| .... | .... |
| butterfly | 34 |

In [27]:
# вставьте код здесь
import pandas as pd
frequency = dict()
for token in tokens:
    if token in frequency.keys():
        frequency[token] += 1
    else:
        frequency[token] = 1

In [36]:
export = pd.DataFrame().from_dict(frequency, orient='index', columns=['Count'])
export.to_csv('./word_frequency.csv', index=True)

#### Шаг №5
Произвести частотный анализ [N-грам](https://en.wikipedia.org/wiki/N-gram) и сохранить в выходной файл в порядке убывания частот (формат файла аналогичен предыдущему примеру, за исключением столбца Word, который заменяется на N-gram. В этой задаче можно использовать готовые библиотеки (смотрим состав библиотеки [NLTK](https://www.nltk.org/)). Формат выходного файла:

| bigram | | Count |
| --- | --- | --- |
| cat | is | 1000 |
| is | a | 300 |
| .... | .... | .... |
| the | fish | 34 |

* Биграм

In [38]:
# вставьте код здесь
bigram_frequency = dict()
for i , token in enumerate(tokens):
    if i == 0:
        continue
    if tokens[i-1] + ' ' + token in bigram_frequency.keys():
        bigram_frequency[tokens[i-1] + ' ' + token] += 1
    else:
        bigram_frequency[tokens[i-1] + ' ' + token] = 1

In [39]:
export = pd.DataFrame().from_dict(bigram_frequency, orient='index', columns=['Count'])
export.to_csv('./bigram_frequency.csv', index=True)

* 3-грам

In [42]:
# вставьте код здесь
trigram_frequency = dict()
for i , token in enumerate(tokens):
    if i == 0 or i == 1:
        continue
    if tokens[i-2] + ' ' + tokens[i-1] + ' ' + token in trigram_frequency.keys():
        trigram_frequency[tokens[i-2] + ' ' + tokens[i-1] + ' ' + token] += 1
    else:
        trigram_frequency[tokens[i-2] + ' ' + tokens[i-1] + ' ' + token] = 1

In [43]:
export = pd.DataFrame().from_dict(trigram_frequency, orient='index', columns=['Count'])
export.to_csv('./trigram_frequency.csv', index=True)

#### Шаг №6 (аналогично шагу 3)
Произвести токенезацию, но уже учитывая знаки препинания:
* С помощью nltk.tokenize.RegexpTokenizer.

In [52]:
# вставьте код здесь
regtokenizer1 = nltk.tokenize.RegexpTokenizer(r"([A-Za-z0-9]+|[!\"#\$%&\\'\(\)\*\+,-\./:;<=>\?@\[\\\\\]\^_`\{\|\}~\.])")
tokens1 = []
for line in data:
    tokens1 += regtokenizer1.tokenize(line)
tokens1[:10]

['chapter', 'i', '"', 'well', ',', 'prince', ',', 'so', 'genoa', 'and']

* С помощью nltk.word_tokenize.

In [45]:
# вставьте код здесь
wordtokens1 = []
for line in data:
    wordtokens1 += nltk.tokenize.word_tokenize(line)
wordtokens1[:10]

['chapter', 'i', '``', 'well', ',', 'prince', ',', 'so', 'genoa', 'and']

#### Шаг №7 (аналогично шагу 4)
Произвести частотный анализ слов.

In [47]:
# вставьте код здесь
frequency1 = dict()
for token in tokens1:
    if token in frequency1.keys():
        frequency1[token] += 1
    else:
        frequency1[token] = 1

In [48]:
export = pd.DataFrame().from_dict(frequency1, orient='index', columns=['Count'])
export.to_csv('./word_frequency_with_punct_tokenizer.csv', index=True)

#### Шаг №8 (аналогично шагу 5)
Произвести частотный анализ N-грам.

* Биграм

In [54]:
# вставьте код здесь
bigram_frequency1 = dict()
for i , token in enumerate(tokens1):
    if i == 0:
        continue
    if tokens1[i-1] + ' ' + token in bigram_frequency1.keys():
        bigram_frequency1[tokens1[i-1] + ' ' + token] += 1
    else:
        bigram_frequency1[tokens1[i-1] + ' ' + token] = 1

In [55]:
export = pd.DataFrame().from_dict(bigram_frequency1, orient='index', columns=['Count'])
export.to_csv('./bigram_frequency_with_punct_tokenizer.csv', index=True)

* 3-грам

In [60]:
# вставьте код здесь
trigram_frequency1 = dict()
for i , token in enumerate(tokens1):
    if i == 0 or i == 1:
        continue
    if tokens1[i-2] + ' ' + tokens1[i-1] + ' ' + token in trigram_frequency1.keys():
        trigram_frequency1[tokens1[i-2] + ' ' + tokens1[i-1] + ' ' + token] += 1
    else:
        trigram_frequency1[tokens1[i-2] + ' ' + tokens1[i-1] + ' ' + token] = 1

In [61]:
export = pd.DataFrame().from_dict(trigram_frequency1, orient='index', columns=['Count'])
export.to_csv('./trigram_frequency_with_punct_tokenizer.csv', index=True)

#### Шаг №9 (используя результаты 5-го шага)
Произвести генерацию текста на основе частот биграм. 

In [58]:
# вставьте код здесь
unused = {(key.split()[0],key.split()[1]) for key in bigram_frequency.keys()}
text = ['i']
for _ in range(40):
    max_ = 0
    max_t = ''  
    for bigram in unused:
        if text[-1] == bigram[0]:
            if bigram_frequency[' '.join(bigram)] > max_:
                max_ = bigram_frequency[' '.join(bigram)]
                max_t = bigram[1]     
    if max_t == '':
        break   
    unused.remove((text[-1],max_t))     
    text.append(max_t)
    



In [59]:
print(' '.join(text))

i am i have been in the same time to the french army and the emperor s face and that the old man who had been sent to be a man s a long time and he had not to him
